In [1]:
import pandas as pd
import numpy as np
import pickle
from scipy import spatial
pd.set_option('display.max_colwidth',3000)

## Exploring Patent Titles

In [2]:
df = pd.read_csv("data/us_pto_2017/documentid_sample.csv")#List of all patents granted since 1970
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 12 columns):
rf_id            100000 non-null int64
title            97487 non-null object
lang             97487 non-null object
appno_doc_num    99645 non-null object
appno_date       97459 non-null object
appno_country    99645 non-null object
pgpub_doc_num    56118 non-null float64
pgpub_date       56118 non-null object
pgpub_country    56118 non-null object
grant_doc_num    82318 non-null object
grant_date       79787 non-null object
grant_country    82318 non-null object
dtypes: float64(1), int64(1), object(10)
memory usage: 9.2+ MB


In [3]:
df.head()

,rf_id,title,lang,appno_doc_num,appno_date,appno_country,pgpub_doc_num,pgpub_date,pgpub_country,grant_doc_num,grant_date,grant_country
0,230320376,Carrier and method,en,12152704,2008-05-15,US,2.008030e+10,2008-12-04,US,NaN,NaN,NaN
1,173100340,VAPOR CHAMBER FOR DISSIPATION HEAT GENERATED BY ELECTRONIC COMPONENT,en,11308231,2006-03-14,US,2.007022e+10,2007-09-20,US,7447029,2008-11-04,US
2,437320689,SECURE INSTANT MESSAGING SYSTEM,en,14713267,2015-05-15,US,2.015026e+10,2015-09-17,US,9509681,2016-11-29,US
3,216570377,NECK WRAP HAVING THERMAL CELLS,en,29087654,1998-05-07,US,NaN,NaN,NaN,D417283,1999-11-30,US
4,259780092,METHOD FOR REMOVING TITANIUM FROM HEXACHLORODISILANE,en,13119349,2011-03-16,US,2.011018e+10,2011-07-28,US,8226919,2012-07-24,US


In [4]:
df['lang'].value_counts()

en    97487
Name: lang, dtype: int64

all in english

In [5]:
df['appno_date'].describe()

count          97459
unique         10580
top       2013-03-15
freq             176
Name: appno_date, dtype: object

In [6]:
pd.to_datetime(df['appno_date']).describe()

count                   97459
unique                  10580
top       2013-03-15 00:00:00
freq                      176
first     1959-01-05 00:00:00
last      2017-10-31 00:00:00
Name: appno_date, dtype: object

In [7]:
pd.to_datetime(df['grant_date']).describe()

count                   79787
unique                   2266
top       2014-04-29 00:00:00
freq                      106
first     1963-03-26 00:00:00
last      2018-02-27 00:00:00
Name: grant_date, dtype: object

In [8]:
pd.to_datetime(df['pgpub_date']).describe()

count                   56118
unique                    883
top       2014-09-18 00:00:00
freq                      253
first     2001-04-05 00:00:00
last      2018-03-01 00:00:00
Name: pgpub_date, dtype: object

In [9]:
df['appno_date'] = pd.to_datetime(df['appno_date'])
df.sort_values('appno_date',inplace=True)
df.dropna(subset=['title'],inplace=True)
df.drop_duplicates(subset=['title'],inplace=True)
df['appno_year'] = df['appno_date'].dt.year
df = df[df['title'] != '?']
#df.set_index('grant_date',inplace=True)
#df.sort_index(inplace=True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92689 entries, 95986 to 91880
Data columns (total 13 columns):
rf_id            92689 non-null int64
title            92689 non-null object
lang             92689 non-null object
appno_doc_num    92689 non-null object
appno_date       92655 non-null datetime64[ns]
appno_country    92689 non-null object
pgpub_doc_num    53867 non-null float64
pgpub_date       53867 non-null object
pgpub_country    53867 non-null object
grant_doc_num    75576 non-null object
grant_date       75540 non-null object
grant_country    75576 non-null object
appno_year       92655 non-null float64
dtypes: datetime64[ns](1), float64(2), int64(1), object(9)
memory usage: 9.9+ MB


In [51]:
df.sample(20)

,rf_id,title,lang,appno_doc_num,appno_date,appno_country,pgpub_doc_num,pgpub_date,pgpub_country,grant_doc_num,grant_date,grant_country,appno_year,topic
70320,193990831,REFRIGERANT MANAGEMENT SYSTEM FOR OPTIMAL COMPRESSOR PERFORMANCE,en,10611135,2003-06-30,US,NaN,NaN,NaN,6834510,2004-12-28,US,2003.0,38
45553,268860969,METHOD FOR MANUFACTURING CIRCUIT BOARD AND METHOD FOR MANUFACTURING STRUCTURE USING THE SAME,en,13094991,2011-04-27,US,2.011031e+10,2011-12-29,US,8863377,2014-10-21,US,2011.0,19
17050,335500774,"ANTENNA CALIBRATION METHOD, SYSTEM AND APPARATUS",en,14379072,2014-08-15,US,2.015034e+10,2015-11-26,US,9419698,2016-08-16,US,2014.0,49
69955,213270932,ORGANOCATALYSTS AND METHODS OF USE IN CHEMICAL SYNTHESIS,en,11546092,2006-10-11,US,2.009009e+10,2009-04-02,US,8084641,2011-12-27,US,2006.0,98
36629,126620430,Personalizing electronic devices and smart covering,en,10087098,2002-03-01,US,2.003002e+10,2003-01-23,US,NaN,NaN,NaN,2002.0,14
70108,348030468,COMPOUNDS AND THEIR USE AS BACE INHIBITORS,en,14486959,2014-09-15,US,2.015013e+10,2015-05-14,US,9248129,2016-02-02,US,2014.0,54
72188,156680391,PORTABLE LIGHT EMITTING DEVICE,en,29204158,2004-04-23,US,NaN,NaN,NaN,D528676,2006-09-19,US,2004.0,16
72506,225420223,Lanthanide-promoted rhodium catalysts and process for producing synthesis gas,en,11028077,2005-01-03,US,2.005011e+10,2005-05-26,US,NaN,NaN,NaN,2005.0,46
18187,148960545,MACHINE CONTROL APPARATUS AND METHOD,en,10755661,2004-01-12,US,2.004026e+10,2004-12-23,US,7389845,2008-06-24,US,2004.0,0
46240,195090430,Topical compositions for anti-aging skin treatment,en,11820426,2007-06-19,US,2.008032e+10,2008-12-25,US,NaN,NaN,NaN,2007.0,8


In [12]:
patent_titles = df['title']

In [13]:
#patent_titles.dropna(inplace=True)

In [14]:
patent_titles.head()

95986             CENTRAL OFFICE MASSIVE MEMORY RECORDING SYSTEM
6477                                            COMPRESSION BAND
82881    DRIVE AND SUSPENSION ARRANGEMENTFOR ARTICULATED VEHICLE
36055         METHOD OF SHIRRING SYNTHETIC TUBLAR SAUSAGE CASING
70801                                                     GIRDLE
Name: title, dtype: object

In [15]:
patent_titles.describe()

count                                                                                                92689
unique                                                                                               92689
top       METHOD AND MACHINE FOR MANUFACTURING ENERGY ACCUMULATING COMPONENTS, AND COMPONENTS MADE THEREBY
freq                                                                                                     1
Name: title, dtype: object

## Vectorization of Patent Titles

In [16]:
from sklearn.feature_extraction import text

## TFIDF

## Custom tokenizing

In [17]:
# additional_stops = ['electr','oper','medium','product','cell','display','liquid','control','make','optic',\
#                     'contain','storag','high','system','devic',\
#                     'manag','machin','process','method','apparatus','use',\
#                     'using','thereof','methods','having','making','based']
additional_stops = ['manufactur','make','contain','storag','high','system','devic',\
                    'manag','process','method','apparatus','use','using','thereof','having','based']


In [18]:
stop_words = text.ENGLISH_STOP_WORDS.union(additional_stops)

In [19]:
import nltk
import string

In [20]:
from nltk.stem.snowball import SnowballStemmer

In [21]:
stemmer = SnowballStemmer('english',ignore_stopwords=True)
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        if item != ',':
            stemmed.append(stemmer.stem(item))
    return stemmed
def tokenize(text):
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

In [22]:
tf = text.TfidfVectorizer(stop_words=stop_words, ngram_range=(1,1), tokenizer=tokenize, max_features=None)

In [23]:
tfidf_vecs = tf.fit_transform(patent_titles)

In [24]:
tfidf_vecs.shape

(92689, 29314)

In [25]:
len(tf.get_feature_names())

29314

## Topic Modeling

In [26]:
def print_topics(topics):
    for i,topic in enumerate(topics):
        print("Topic {}".format(i))
        print(", ".join(topic)+"\n")

## NMF

In [27]:
from sklearn.decomposition import NMF
nmf = NMF(n_components=100, random_state=42)
nmf_vecs = nmf.fit_transform(tfidf_vecs)

In [28]:
nmf.components_.shape

(100, 29314)

In [29]:
with open('nmf_patent_vecs.pkl','wb') as file:
    pickle.dump(nmf_vecs,file)

In [30]:
#len(df_vecs.columns)

### topics

In [31]:
features = tf.get_feature_names()

In [32]:
topics = [[features[idx] for idx in np.flip(np.argsort(component)[-100:])] for component in nmf.components_]

In [33]:
print_topics(topics)

Topic 0
control, remot, flow, temperatur, speed, motor, gain, emiss, robot, brake, medium, therefor, hydraul, rate, steer, output, oscil, time, releas, fan, environment, numer, refriger, input, variabl, processor, current, feedback, shift, voltag, clutch, hybrid, loop, level, ratio, programm, industri, traffic, charg, congest, torqu, forc, synchron, throttl, motion, transmitt, load, dynam, airfuel, respons, mean, uplink, bus, rotat, mode, gestur, driver, cruis, manipul, plural, evapor, servo, aircraft, ac, particl, induct, applianc, channel, movement, cursor, logic, foam, bicycl, pest, electrostat, consol, elev, humid, direct, ventil, admiss, smart, plant, proport, qualiti, strategi, autonom, beam, defrost, function, convert, idl, thermostat, actuat, paramet, water, deceler, central, refresh, inject

Topic 1
imag, captur, pickup, solidst, color, medic, read, len, scan, correct, medium, radiat, develop, reson, sens, ultrasound, xray, compress, decod, pixel, ultrason, project, toner, enh

In [34]:
# topic_titles = ['Semiconductor Manufacturing','Liquid Crystal Displays',\
#                    'Image Processing','Engines and Controls','Mechanical Vehicle Parts','Integrated Circuits',
#             'LEDs','Data Storage and Processing',]

In [35]:
len(df)

92689

In [36]:
nmf_vecs[24]

array([2.54827597e-05, 0.00000000e+00, 2.51042923e-04, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.41755245e-05,
       1.00841945e-04, 0.00000000e+00, 1.56944967e-04, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.71273170e-04, 0.00000000e+00, 3.52960958e-04,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 7.35934324e-04,
       2.05103508e-04, 0.00000000e+00, 1.58440553e-04, 0.00000000e+00,
       1.45843772e-04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       3.61253632e-04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 4.04038020e-05, 1.10101546e-04, 4.87455934e-05,
       0.00000000e+00, 0.00000000e+00, 1.10605910e-04, 6.76383189e-05,
       8.80806530e-05, 0.00000000e+00, 8.70284965e-05, 0.00000000e+00,
       3.18598570e-05, 0.00000000e+00, 2.70065292e-06, 0.00000000e+00,
       7.08205455e-04, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [37]:
df['topic']=[nmf_vec.argmax() for nmf_vec in nmf_vecs]

In [38]:
df['topic'].value_counts()

95    3287
8     2693
98    1959
6     1931
72    1930
9     1925
54    1839
1     1754
12    1640
15    1618
85    1603
96    1458
63    1424
0     1408
92    1379
42    1344
66    1298
14    1226
33    1224
99    1217
86    1189
89    1170
64    1159
41    1156
87    1140
18    1137
16    1110
84    1109
73    1099
79    1081
      ... 
29     650
45     647
82     607
13     590
35     583
90     575
44     564
28     558
30     549
38     548
77     534
5      521
74     520
51     519
88     513
24     482
57     481
25     461
39     452
26     443
23     428
10     415
76     410
20     387
48     326
65     321
43     306
47     304
60     301
40     173
Name: topic, Length: 100, dtype: int64

In [39]:
#patent_titles[df['topic']==9].sample(300)

In [43]:
patent_topic_vec_dict = {appno_doc_num:nmf_vecs[i] for i,appno_doc_num in enumerate(df.index)}

In [55]:
def recommend_patent(appno_doc_num):
    #get cosine distance from each patent
    patent_vec_a = patent_topic_vec_dict[appno_doc_num]
    distances = {appno_doc_num_b:spatial.distance.cosine(patent_vec_a,patent_vec_b) for appno_doc_num_b,patent_vec_b in patent_topic_vec_dict.items()}
    return [df.loc[key]['title'] for key in sorted(distances, key=distances.get)[:4]],[distances[key] for key in sorted(distances, key=distances.get)[:4]]

In [56]:
recommend_patent(49044)

/anaconda3/lib/python3.6/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


(['METHODS FOR THE SYNTHESIS OF ASTAXANTHIN',
  'SYNTHESIS OF N-[N-(3,3-DIMETHYLBUTYL)-L-ALPHA-ASPARTYL]-L-PHENYLALANINE 1-METHYL ESTER USING L-ALPHA-ASPARTYL-L- PHENYLALANINE 1-METHYL ESTER PRECURSORS',
  'SMALL-MOLECULE INHIBITORS OF PROTEIN SYNTHESIS INACTIVATING TOXINS',
  'PHOSPHONATE HERBICIDAL ANTIDOTES'],
 [0.0019921623116306852,
  0.05571108339849795,
  0.0847439023950014,
  0.08705154996924214])